In [17]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%matplotlib qt

# Busca incremental e busca adaptativa

## $ \S 1 $ Descrição da busca incremental para encaixotamento de raízes

Suponha que queiramos encaixotar um zero de uma função contínua $ f \colon [a,
b] \to \mathbb{R} $. Usando o método da __busca incremental__, começamos
escolhendo um __incremento__ ou __tamanho de passo__ $ h > 0 $. Então calculamos
o sinal de
$$
f(x_i) \quad \text{para} \quad x_i = a+ih \quad (i = 0, 1, \dots )
$$
sucessivamente.
Se 
$$
    \operatorname{sinal} f(x_i) \ne \operatorname{sinal} f(x_{i+1})
$$
para algum $ i $, então o subintervalo $ [x_i,x_{i+1}] $ deve conter um zero de
$ f $, pelo teorema do valor intermediário. Caso esta condição falhe para todo $
i $, eventualmente teremos $ x_{i+1} > b $ e a busca terá sido _inconclusiva_.

__Exemplo 1.1:__ Use a busca incremental com tamanho de passo $ 1 $ para tentar
encontrar um zero do polinômio $ f(x) = x^3 - 6x^2 + 11x - 7 $ no intervalo $
[0, 5] $.

_Solução:_ Precisamos avaliar $ f $ nos pontos $ 0,\,1,\,2,\,3,\,4 $ e $ 5 $. Vamos
usar Python como nossa calculadora.


In [1]:
f = lambda x: x**3 - 6 * x**2 + 11 * x - 7
for n in range(0, 6):
    print(f"O valor de f em x = {n} é {f(n)} .")

O valor de f em x = 0 é -7 .
O valor de f em x = 1 é -1 .
O valor de f em x = 2 é -1 .
O valor de f em x = 3 é -1 .
O valor de f em x = 4 é 5 .
O valor de f em x = 5 é 23 .



| Intervalo | $f(x)$ na extrem. esq. | $f(x)$ na extrem. dir. |Troca de sinal?|
|-----------|------------------------|------------------------|---------------|
| $[0, 1]$  | $-7$                   | $-1$                   | Não           |
| $[1, 2]$  | $-1$                   | $-1$                   | Não           |
| $[2, 3]$  | $-1$                   | $-1$                   | Não           |
| $[3, 4]$  | $-1$                   | $5$                    | Sim           |
| $[4, 5]$  | $5$                    | $23$                   | Não           |

Concluímos que $ f $ possui ao menos um zero no intervalo $ [3, 4] $. 

__Problema 1:__ Utilize a busca incremental com o tamanho de passo indicado para
encontrar um ou mais subintervalos que encaixotem um zero das funções seguintes:

(a) $ g(x) = \sin x + x^2 -  x - 1 $ no intervalo $ [0, 2\pi] $, com tamanho de passo $ \frac{\pi}{2} $.

(b) $ h(x) = x^4 - 4x^3 - 8x^2 + 48x - 21 $ no intervalo $ [0, 5] $, com tamanho de passo $ 1 $.

(c) $ f(x) = e^{-x} - x $ no intervalo $ [0, 1] $, com tamanho de passo $ 0.1 $.

# $ \S 2 $ Implementação da busca incremental

In [4]:
def incremental_search(f: Callable[[float], float],
                       a: float, b: float, h: float
                       ) -> tuple[float, float]:
    """
    Beginning with x_0 = a and x_1 = a + h and repeatedly incrementing both by
    h, returns the first pair of consecutive nodes where f takes on opposite
    signs.
    Parameters:
        * A real continuous function f.
        * The endpoints a < b of an interval [a, b] where f is defined.
        * A step size h > 0. 
    Output:
        * In case of success: updated values of a, b such that a < b, 
          b - a <= h and f(a)f(b) < 0. In case of failure: None, None.
    Prints:
        * The number of performed iterations.
        * The pair a, b described above in case of success;
          or a warning message in case of failure.
    """
    from numpy import sign
    

    # Check the sign of h:
    if sign(h) != 1:
        raise ValueError("The step size should be positive!")
    # Check the sign of (b - a):
    if a >= b:
        raise ValueError("a should be less than b!")
    # Initializing:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    iterations = 1
    
    while sign(f_0) == sign(f_1) and x_0 != b:
        x_0, f_0 = x_1, f_1         # Update x_0, f_0 to their next values.
        x_1 = min(x_1 + h, b)       # Update x_1.
        f_1 = f(x_1)                # Update f_1.
        iterations += 1
    
    print(f"The number of performed iterations was: {iterations}.")
    if x_0 == b:                    # The right endpoint was reached.
        print("The search was inconclusive! You may want to try again "
              "with a smaller step size or a different interval.")
        return None, None
    else:
        print("The search was successful! The function f changes sign "
              "in the subinterval [a, b] where:"
              f"\n\t a = {x_0:15.9f}"
              f"\n\t b = {x_1:15.9f}")
        return x_0, x_1

__Problema 2:__ O polinômio $ x^3 - 11x^2 + 7 $ possui exatamente um zero entre $ 0 $ e $ 1 $. Encaixote este zero dentro de um intervalo de comprimento no máximo $ 10^{-3} $.

*Solução:*

In [7]:
f = lambda x: x**3 - 11 * x**2 + 7
a = 0
b = 1
h = 1.0e-3
a, b = incremental_search(f, a, b, h)
print(a, b)

The number of performed iterations was: 830.
The search was successful! The function f changes sign in the subinterval [a, b] where:
	 a =     0.829000000
	 b =     0.830000000
0.8290000000000006 0.8300000000000006


⚠️ Se $ f > 0 $ ou se $ f < 0 $ em todo o intervalo $ [a,b] $, obviamente a
busca incremental será mal-sucedida. Por outro lado, mesmo que $ f $ troque de
sinal dentro de $ [a, b] $, por menor que seja o tamanho do incremento $ h $,
não há como garantir _a priori_ que a busca será bem-sucedida, ou que o
intervalo resultante contenha um _único_ zero.

️️📝 Muitas vezes precisamos encontrar _todos_ os zeros de uma função. Nestes
casos, se utilizarmos um incremento $ h $ grande demais, corremos o risco de
pular por um número par de zeros consecutivos sem detectá-los, ou de obter um
intervalo que contém um número ímpar de zeros em seu interior, mas ao final
conseguir localizar apenas um deles. Por outro lado, se $ h $ for pequeno
demais, gastaremos muito tempo procurando em regiões que não contêm qualquer
zero. Cabe ao usuário escolher o tamanho de passo $ h $ mais adequado em cada
caso de modo a mitigar estas dificuldades.

__Problema 3:__

(a) Quantos zeros a função $ f(x) = \sin(2x) - 0.999 $ tem no intervalo $ [0, 30\pi] $?

(b) Mostre que `busca_incremental` com tamanho de passo $ h = 1 $ não consegue encaixotar nenhum deles.

*Solução:*

In [ ]:
from numpy import sin, pi

## $ \S 3 $ Análise do desempenho da busca incremental

**Definição:** As funções

\begin{alignat*}{9}
    \lfloor{\cdot}\rfloor \colon \mathbb{R} \to \mathbb{Z},
    \quad \lfloor{x}\rfloor & = \text{maior inteiro $ \le x $} \\
    \lceil{\cdot}\rceil \colon \mathbb{R} \to \mathbb{Z},
    \quad \lceil{x}\rceil & = \text{menor inteiro $ \ge x $} \\
\end{alignat*}
são chamadas de funções **chão** e **teto** respectivamente. Observe que
$$
  \lfloor x \rfloor = x = \lceil x \rceil\ \Longleftrightarrow x \in \mathbb Z\,.
$$


__Problema 4:__ 

(a) Esboce os gráficos das funções chão e teto, primeiro à mão e depois usando o computador.

(b) Mostre que estas funções são contínuas exceto nos inteiros, onde têm uma descontinuidade do tipo salto.


_Solução:_

In [16]:
from numpy import floor, ceil
import matplotlib.pyplot as plt


# Complete o código abaixo:
xs = np.linspace(-5, 5, 1001)
ys_floor = ...
ys_ceil = ...
plt.plot(...)
plt.plot(...)
plt.grid(True)
plt.show()

__Problema 5:__ Mostre que
$$
\lceil{x}\rceil =
\begin{cases}
    \lfloor{x}\rfloor + 1 & \text{se $ x \not \in \mathbb{Z} $} \\
    \lfloor{x}\rfloor & \text{se $ x \in \mathbb{Z} $}
\end{cases}
$$


_Solução:_

__Teorema 3.1:__ _No pior caso, a busca incremental aplicada a uma função
definida no intervalo $ [a, b] $ com tamanho de passo $ h > 0 $
requer_
\begin{equation}
    \boxed{n = \left\lceil{\frac{b-a}{h}}\right\rceil + 1 \ \text{avaliações}.}
    \tag{1}
\end{equation}

__Prova:__ Suponha que a função à qual a busca foi aplicada não troque de sinal
dentro de $ [a, b] $. Então precisamos avaliá-la em cada ponto $ x_i = a + ih $
tal que $ x_i < b $ a partir de $ i = 0 $, e também em $ b $. Portanto:
* Se $ h $ divide $ b - a $ exatamente, é necessária uma avaliação da
  função para cada inteiro $ i $ entre $ 0 $ e $ \frac{b - a}{h} $.
* Se $ h $ não divide $ b - a $ exatamente, precisamos realizar uma avaliação
  da função para cada inteiro $ i $ entre $ 0 $ e
  $ \left\lfloor \frac{b - a}{h} \right\rfloor $, além de $ b $.

Em qualquer caso, o número de avaliações coincide com aquele do enunciado.
<div style="text-align: right">$ \blacksquare $ </div>

Por exemplo, se quisermos localizar o zero de uma função definida no intervalo
$ [0, 1] $ com precisão de ao menos $ 5 $ dígitos, precisamos realizar a
princípio $ 10^5 $ avaliações da função. Assim, para uma tolerância
pequena relativamente ao comprimento do intervalo original, o custo
computacional pode ser inaceitavelmente alto.


__Problema 6:__ Usando a busca incremental, identifique um intervalo $ [a, b] $
de comprimento menor que $ \frac{1}{10} $ que contém uma raiz da equação dada.
Determine em cada caso uma cota superior para o número de passos necessários
através do Teorema 3.1.

(a) $ x \ln x = 1 $ ($ x > 0 $).

(b) $ \cos x = x^2 $.

(c) $ x^5 - 3x^4 - 6x^3 + 4x^2 + 5x - 3 = 0 $.

(d) $ \tan x = x + 2 e^x $ $\big( x \not\in \frac{\pi}{2} + \pi \mathbb Z \big)$.

_Solução:_

## $ \S 4 $ Descrição da busca adaptativa

Uma alternativa para tentar reduzir o custo da busca incremental é utilizar um
tamanho de passo que é reduzido pouco a pouco. Mais precisamente, na __busca
adaptativa__ com tolerância $ \varepsilon > 0 $:

1. Inicialmente tomamos
$$
h = \frac{b-a}{10}\,.
$$
2. Realizamos uma busca incremental no intervalo $ [a, b] $ com tamanho de passo $ h $.
   Se ela for bem-sucedida, atualizamos $ [a, b] $ a este
   subintervalo. Caso contrário, não alteramos $ [a, b] $.
3. Se $ h < \varepsilon $, terminamos. Caso contrário, fazemos $ h \leftarrow h / 10 $
   e voltamos ao passo 2.

O ponto crucial é que cada vez que encontrarmos um subintervalo onde a função
troca de sinal, podemos restringir a próxima busca incremental a este intervalo
menor, potencialmente economizando o número de avaliações realizadas.

## $ \S 5 $ Implementação da busca adaptativa

In [4]:
def adaptive_search(f: Callable[[float], float],
                    a: float, b: float, h_max: float
                    ) -> tuple[float, float]:
    """
    Initially an incremental search is performed on [a, b] with a step size
    h = (b - a) / 10. If the search is successful, [a, b] is updated to the
    interval that was yielded. Otherwise it remains the same. In any case, we
    again perform an incremental search, this time with a step of size one-tenth
    the size of the previous one. This is repeated until h <= h_max / 10.
    Parameters:
        * A real continuous function f.
        * The endpoints a and b of an interval [a, b] where f is defined.
        * An upper bound h_max for the desired step size.
    Output:
        * In case of success: updated values of a, b such that a < b,
          b - a <= h_max and f(a)f(b) < 0. In case of failure: None, None.
    Prints:
        * The pair a, b described above in case of success;
          or a warning message in case of failure.
        * The number of performed iterations.
    """
    from numpy import sign


    # Check the sign of h_max:
    if sign(h_max) != 1:
        raise ValueError("The tolerance should be positive!")
    # Check the sign of (b - a):
    if a >= b:
        raise ValueError("a should be less than b!")
    # Check whether h_max < (b - a) / 10:
    if h_max > (b - a) / 10:
        h_max = (b - a) / 10
    # Initializing:
    iterations = 1
    h =  (b - a) / 10
    success = False
    
    while h >= h_max:
        x_0 = a
        x_1 = x_0 + h
        f_0 = f(x_0)
        f_1 = f(x_1)
        while sign(f_0) == sign(f_1) and x_0 != b:
            x_0, f_0 = x_1, f_1         # Update x_0, f_0 to their next values.
            x_1 = min(x_1 + h, b)       # Update x_1.
            f_1 = f(x_1)                # Update f_1.
            iterations += 1
        if x_0 != b:
            success = True
            a = x_0
            b = x_1
        h /= 10

    print(f"The number of performed iterations was: {iterations}.")
    if not success:                     # Could not find a change of sign.
        print("The search was inconclusive! You may want to try again "
              "with a smaller tolerance or a different interval.")
        return None, None
    else:
        print("The search was successful! The function f changes sign "
              "in the subinterval [a, b] where:"
              f"\n\t a = {x_0:15.9f}"
              f"\n\t b = {x_1:15.9f}")
        return x_0, x_1

⚠️ Não há garantia que a busca adaptativa seja sempre mais eficiente que a busca
incremental simples. Por exemplo, se a função toma valores de mesmo sinal ao
longo de todo o intervalo inicial $ [a, b] $, então para $ h < \frac{b - a}{10}
$, a busca adaptativa certamente realizará mais avaliações que a busca incremental.


__Problema 7:__ Compare o número de iterações utilizadas pela busca incremental
e pela busca adaptativa para encaixotar uma raiz das equações seguintes para
o valor máximo do incremento $ h $ igual a $ 10^{-3} $.

(a) $ \cos x \cosh x = 2 $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

(b) $ xe^x = 1 $.

(c) $ x^2 + \ln x = 0 $ ($ x > 0 $).

_Solução:_

__Problema 8:__ Modifique `busca_incremental` para criar um novo procedimento `busca_exaustiva` com os mesmos parâmetros que antes, mas que retorna uma lista contendo todos os pares $ \big(x_{i-1}, x_i\big) $ tais que ocorre uma troca de sinal no intervalo com estas extremidades, onde
$$
    x_i = a + h i \quad \text{para} \quad i = 0,1,\dots,N - 1 =\left\lfloor{\frac{b-a}{h}}\right\rfloor \quad\text{e} \quad x_N = b.
$$

*Solução:*